In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import NASNetLarge, ResNet101, DenseNet121
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.metrics import Precision, Recall
from tqdm.keras import TqdmCallback
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, ModelCheckpoint

from keras.models import load_model

print(f'tf version - {tf.__version__}')

In [ ]:
IMAGE_PATH = '../input/plant-pathology-2021-224x224/train_imgs'

In [ ]:
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
train

In [ ]:
len(train.labels.unique())

In [ ]:
from PIL import Image
Image.open(os.path.join('../input/plant-pathology-2021-fgvc8/train_images', train[train.labels == 'rust'].image.iloc[0]))

In [ ]:
train.labels.value_counts()

In [ ]:
CLASSES = train['labels'].unique().tolist()
BATH_SIZE = 128
EPOCH = 10

In [ ]:
data_generator = ImageDataGenerator(
    validation_split=0.15,
)

In [ ]:
train_data_loader = data_generator.flow_from_dataframe(
    train,
    directory=IMAGE_PATH,
    classes=CLASSES,
    x_col="image",
    y_col="labels",
    target_size=(150, 150),
    subset='training'
)

val_data_loader = data_generator.flow_from_dataframe(
    train,
    directory=IMAGE_PATH,
    classes=CLASSES,
    x_col="image",
    y_col="labels",
    target_size=(150, 150),
    subset='validation'
)

In [ ]:
dict_classes = train_data_loader.class_indices
dict_classes

In [ ]:
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=0.001)

CALLBACKS = [ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=0.5, factor=0.5),
             EarlyStopping(monitor='val_loss', verbose=1, patience=5),
             TqdmCallback(verbose=0)]

In [ ]:
base_resnet = ResNet101(
                    weights='../input/keras-pretrained-models/ResNet101_NoTop_ImageNet.h5',
                    include_top=False, 
                    pooling='avg', 
                    input_shape=(150, 150, 3))

model_resnet = Sequential()
model_resnet.add(base_resnet)
model_resnet.add(layers.Dense(12, activation='softmax'))

model_resnet.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy', 'Precision', 'Recall'])
model_resnet.summary()

history_resnet = model_resnet.fit(train_data_loader, 
                            validation_data=val_data_loader, 
                            batch_size=BATH_SIZE,
                            epochs=9,
                            callbacks=CALLBACKS)

In [ ]:
base_DenseNet121 = tf.keras.applications.DenseNet121(
                    weights='../input/keras-pretrained-models/DenseNet121_NoTop_ImageNet.h5',
                    include_top=False, 
                    pooling='avg', 
                    input_shape=(150, 150, 3))

model_DenseNet121 = Sequential()
model_DenseNet121.add(base_DenseNet121)
model_DenseNet121.add(layers.Dense(12, activation='softmax'))

model_DenseNet121.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])
model_DenseNet121.summary()

history_DenseNet121 = model_DenseNet121.fit(train_data_loader, 
                                  validation_data=val_data_loader, 
                                  batch_size=BATH_SIZE,
                                  epochs=15,
                                  callbacks=CALLBACKS)

In [ ]:
base_MobileNetV2 = tf.keras.applications.MobileNetV2(
                    weights='../input/keras-pretrained-models/MobileNetV2_NoTop_ImageNet.h5',
                    include_top=False, 
                    pooling='avg', 
                    input_shape=(150, 150, 3))

model_MobileNetV2 = Sequential()
model_MobileNetV2.add(base_MobileNetV2)
model_MobileNetV2.add(layers.Dense(12, activation='softmax'))

model_MobileNetV2.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])
model_MobileNetV2.summary()

history_MobileNetV2 = model_MobileNetV2.fit(train_data_loader, 
                                  validation_data=val_data_loader, 
                                  batch_size=BATH_SIZE,
                                  epochs=15,
                                  callbacks=CALLBACKS)

In [ ]:
import statistics
from statistics import mode


def most_common(predict_list: list):
    return(mode(predict_list))


def prepare_predicts(predicts: list):
    for i in range(len(predicts)):
        predicts[i] = np.argmax(predicts[i])
    return predicts


test_dir = '/kaggle/input/plant-pathology-2021-fgvc8/test_images/'
test_df = pd.DataFrame({'image':os.listdir(test_dir)})

test_set = data_generator.flow_from_dataframe(dataframe=test_df,
                                    directory=test_dir,
                                    x_col="image",
                                    y_col=None,
                                    batch_size=64,
                                    seed=42,
                                    shuffle=False,
                                    class_mode=None,
                                    target_size=(150,150))

pred_resnet = model_resnet.predict(test_set).tolist()
pred_densnet = model_DenseNet121.predict(test_set).tolist()
pred_mobile = model_MobileNetV2.predict(test_set).tolist()

pred_resnet = prepare_predicts(pred_resnet)
pred_densnet = prepare_predicts(pred_densnet)
pred_mobile = prepare_predicts(pred_mobile)
                                                               

res = []
 
for i in range(len(pred_resnet)):
    try:
        res.append(most_common([pred_resnet[i], pred_densnet[i], pred_mobile[i]]))
    except:
        res.append(pred_resnet[i])

    
def get_key(val):
    for key, value in dict_classes.items():
        if val == value:
            return key
        

for i in range(len(res)):
    res[i] = get_key(res[i])

In [ ]:
test_df['labels'] = res
test_df

In [ ]:
test_df.to_csv('submission.csv', index=False)